In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/melting-point/sample_submission.csv
/kaggle/input/melting-point/train.csv
/kaggle/input/melting-point/test.csv


In [2]:
pip install rdkit-pypi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 45.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
#Import Bibliothèque
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors
from tqdm import tqdm

In [4]:
#Importation des fichiers csv 
folder_path = "/kaggle/input/melting-point"
train_csv_path = '/kaggle/input/melting-point/train.csv'
train_csv = pd.read_csv(train_csv_path)
#print(train_csv.head())

test_csv_path = '/kaggle/input/melting-point/test.csv'
test_csv = pd.read_csv(test_csv_path)
#print(test_csv.head())

In [5]:
#Augmentation des données
train_data_aug = train_csv

# --- paramètres ---
n_aug = 5              # nombre de SMILES augmentés par molécule
smiles_col = "SMILES"  # nom de ta colonne SMILES

# --- fonction de génération de SMILES aléatoires ---
def randomize_smiles(smi, n_aug=5):
    mol = Chem.MolFromSmiles(smi)
    if mol is None:
        return []
    smiles = set()
    for _ in range(n_aug):
        smiles.add(Chem.MolToSmiles(mol, doRandom=True))
    return list(smiles)

# --- génération des données augmentées ---
aug_rows = []

print(f"🔁 Génération de {n_aug} SMILES augmentés par molécule...")
for _, row in tqdm(train_data_aug.iterrows(), total=len(train_data_aug)):
    base_smi = row[smiles_col]
    new_smiles = randomize_smiles(base_smi, n_aug=n_aug)
    for s in new_smiles:
        new_row = row.copy()
        new_row[smiles_col] = s
        aug_rows.append(new_row)

# --- concaténation ---
train_data_aug = pd.concat([train_data_aug, pd.DataFrame(aug_rows)], ignore_index=True)

print(f"✅ Jeu de données étendu : {len(train_csv)} → {len(train_data_aug)} lignes")

train_data_set = train_data_aug

🔁 Génération de 5 SMILES augmentés par molécule...


100%|██████████| 2662/2662 [00:02<00:00, 1253.03it/s]


✅ Jeu de données étendu : 2662 → 14540 lignes


In [6]:
#Chargement du Dataset complet
#Full dataset loading
#train_data_set = train_csv

y = train_data_set['Tm']
list_columns = train_data_set.columns.tolist()
feature_names = list_columns[3:]

# print the list of columns in the dataset to find the name of the prediction target
#train_data_set.describe()

test_data_set = test_csv

In [7]:
#Retraitement des données sous la forme de 0 et 1 - Train
X_df = pd.DataFrame(train_data_set[feature_names])

#Créer les colonnes binaires
binary_feats = {f"has_{col}": (X_df[col] > 0).astype(int) for col in feature_names}

#Ratios 
ratio_feats = {f"{col}_ratio": X_df[col] / (X_df[feature_names].sum(axis=1) + 1e-6) for col in feature_names}

#Totaux globaux
extra_feats = {
    "total_groups": X_df[feature_names].sum(axis=1),
    "nb_groups_non_zero": (X_df[feature_names] > 0).sum(axis=1),
}

# 5. Fusionner toutes les nouvelles colonnes d’un coup
X_df = pd.concat([X_df, pd.DataFrame({**binary_feats, **ratio_feats, **extra_feats})], axis=1)

print("Shape après feature engineering:", X_df.shape)

Shape après feature engineering: (14540, 1274)


In [8]:
#Traitement des SMILES
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors, AllChem

X_smiles = pd.DataFrame(train_data_set["SMILES"])

def compute_chem_features(df, smiles_col="smiles", add_fingerprint=True, fp_nbits=2048):
    """
    Prend un DataFrame avec une colonne de SMILES et renvoie un DataFrame de descripteurs chimiques.
    - df : DataFrame d'entrée (ne modifie pas df).
    - smiles_col : nom de la colonne SMILES.
    - add_fingerprint : bool, calcule fingerprint Morgan (binaire) si True.
    - fp_nbits : taille du vecteur fingerprint.
    """
    records = []
    invalid_idx = []
    
    # SMARTS motifs utiles (exemples)
    smarts = {
        "has_OH": "[OX2H]",              # alcool phénol etc. (H-bond donors)
        "has_carbonyl": "[CX3]=O",      # carbonyle
        "has_nitro": "[NX3](=O)=O",     # nitro
        "has_amine": "[NX3;!$(N-*=O)]", # amine primaire/secondaire/tertiaire (élimine nitro)
        "has_ester": "[$([CX3](=O)O)]",  # ester
        "has_thiol": "[SX2H]",           # thiol
    }
    smarts_compiled = {k: Chem.MolFromSmarts(v) for k, v in smarts.items()}
    
    for idx, smi in df[smiles_col].items():
        mol = None
        try:
            mol = Chem.MolFromSmiles(smi)
            if mol is None:
                # try sanitize false then sanitize
                mol = Chem.MolFromSmiles(smi, sanitize=False)
                if mol is not None:
                    Chem.SanitizeMol(mol)
        except Exception as e:
            mol = None
        
        if mol is None:
            invalid_idx.append(idx)
            # fill NaNs for invalid
            records.append({"index": idx, "valid_smiles": False})
            continue
        
        r = {"index": idx, "valid_smiles": True}
        
        # Atom counts (heavy atoms, heteroatoms, and halogens)
        atom_counts = {}
        for atom in mol.GetAtoms():
            sym = atom.GetSymbol()
            atom_counts[sym] = atom_counts.get(sym, 0) + 1
        # common atoms
        for at in ["C","H","O","N","F","Cl","Br","I","S","P"]:
            r[f"atom_count_{at}"] = atom_counts.get(at, 0)
        r["heavy_atom_count"] = rdMolDescriptors.CalcNumHeavyAtoms(mol)
        
        # Basic descriptors
        r["MolWt"] = Descriptors.ExactMolWt(mol)           # masse exacte
        r["MolLogP"] = Descriptors.MolLogP(mol)            # octanol-water partition
        r["MolMR"] = Descriptors.MolMR(mol)                # molar refractivity (approx)
        r["TPSA"] = rdMolDescriptors.CalcTPSA(mol)         # topological polar surface area
        r["NumHDonors"] = rdMolDescriptors.CalcNumHBD(mol)
        r["NumHAcceptors"] = rdMolDescriptors.CalcNumHBA(mol)
        r["NumRotatableBonds"] = rdMolDescriptors.CalcNumRotatableBonds(mol)
        r["NumRings"] = rdMolDescriptors.CalcNumRings(mol)
        r["NumAromaticRings"] = rdMolDescriptors.CalcNumAromaticRings(mol)
        r["FractionCSP3"] = Descriptors.FractionCSP3(mol)
        r["NumValenceElectrons"] = sum([a.GetTotalValence() for a in mol.GetAtoms()])
        r["FormalCharge"] = Chem.GetFormalCharge(mol)
        
        # ring/aromaticity / complexity
        r["BertzCT"] = Descriptors.BertzCT(mol)             # complexité topologique
        # optionally: r["BalabanJ"] = Descriptors.J(mol)
        
        # SMARTS motif counts/presence
        for name, patt in smarts_compiled.items():
            try:
                matches = mol.GetSubstructMatches(patt)
                r[f"{name}_count"] = len(matches)
                r[f"{name}_present"] = int(len(matches) > 0)
            except Exception:
                r[f"{name}_count"] = 0
                r[f"{name}_present"] = 0
        
        # hydrogen bonding index (simple)
        r["hbond_index"] = r["NumHDonors"] * r["NumHAcceptors"]
        
        # aromatic proportion
        r["aromatic_atom_fraction"] = sum(1 for a in mol.GetAtoms() if a.GetIsAromatic()) / max(1, mol.GetNumAtoms())
        
        # Morgan fingerprint (ECFP) as bit vector (optional)
        if add_fingerprint:
            try:
                fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=fp_nbits)
                arr = np.zeros((1,), dtype=np.int8)
                # convert to numpy array
                onbits = list(fp.GetOnBits())
                # We'll create sparse representation: store as list of 0/1 ints in columns fp_0..fp_nbits-1
                for i in range(fp_nbits):
                    r[f"fp_{i}"] = 1 if i in onbits else 0
            except Exception:
                # fallback: no fp
                for i in range(fp_nbits):
                    r[f"fp_{i}"] = 0
        
        records.append(r)
    
    feats = pd.DataFrame(records).set_index("index")
    # Convert types and fill NaN for invalid SMILES
    feats = feats.sort_index()
    feats = feats.fillna(np.nan)
    
    # Optionally drop fp columns if too heavy
    return feats, invalid_idx

# Utilisation :
feats_df, invalids = compute_chem_features(X_smiles, smiles_col="SMILES", add_fingerprint=False)
print("Invalid SMILES indices:", invalids)
X_df = pd.concat([X_df, pd.DataFrame({**feats_df})], axis=1)
print("Shape après feature engineering chimique:", X_df.shape)
print(X_df.head())

Invalid SMILES indices: []
Shape après feature engineering chimique: (14540, 1313)
   Group 1  Group 2  Group 3  Group 4  Group 5  Group 6  Group 7  Group 8  \
0        0        0        0        0        0        0        0        0   
1        0        0        0        0        0        0        0        0   
2        2        1        0        0        0        0        0        0   
3        1        0        0        0        0        0        0        0   
4        2        3        0        0        0        0        0        0   

   Group 9  Group 10  ...  has_nitro_count  has_nitro_present  \
0        0         0  ...                0                  0   
1        0         0  ...                0                  0   
2        0         0  ...                0                  0   
3        0         0  ...                0                  0   
4        0         0  ...                0                  0   

   has_amine_count  has_amine_present  has_ester_count  has_est

In [9]:
#Retraitement du fichier Test
X_test_df = pd.DataFrame(test_data_set[feature_names])
X_test_smiles = pd.DataFrame(test_data_set["SMILES"])
#Créer les colonnes binaires
binary_feats_test = {f"has_{col}": (X_test_df[col] > 0).astype(int) for col in feature_names}
ratio_feats_test = {f"{col}_ratio": X_test_df[col] / (X_test_df[feature_names].sum(axis=1) + 1e-6) for col in feature_names}
extra_feats_test = {
    "total_groups": X_test_df[feature_names].sum(axis=1),
    "nb_groups_non_zero": (X_test_df[feature_names] > 0).sum(axis=1),
}
X_test_df = pd.concat([X_test_df, pd.DataFrame({**binary_feats_test, **ratio_feats_test, **extra_feats_test})], axis=1)
print("Shape après feature engineering:", X_test_df.shape)
feats_df, invalids = compute_chem_features(X_test_smiles, smiles_col="SMILES", add_fingerprint=False)
X_test_df = pd.concat([X_test_df, pd.DataFrame({**feats_df})], axis=1)
print("Shape après feature engineering chimique:", X_test_df.shape)


Shape après feature engineering: (666, 1274)
Shape après feature engineering chimique: (666, 1313)


In [10]:
X = X_df
#print(X.head())
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

#Split Train / Valid
X_train, X_valid, y_train, y_valid = train_test_split( X, y, test_size=0.2, random_state=42 )


**Import et Selection du modèle**

In [11]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
from catboost import CatBoostRegressor
#from lightgbm import LGBMRegressor
#from xgboost import XGBRegressor

In [12]:
final_model = CatBoostRegressor(
    iterations=2000,
    learning_rate=0.09586251212066181,
    depth=10,
    l2_leaf_reg=3,
    bagging_temperature=2.031454661334689,
    random_state=42,
    verbose=100
)
final_model.fit(X, y)

0:	learn: 80.4605960	total: 123ms	remaining: 4m 5s
100:	learn: 31.1587222	total: 6.08s	remaining: 1m 54s
200:	learn: 24.1014336	total: 12s	remaining: 1m 47s
300:	learn: 20.2133737	total: 17.9s	remaining: 1m 40s
400:	learn: 17.8420393	total: 23.8s	remaining: 1m 34s
500:	learn: 16.3098451	total: 29.7s	remaining: 1m 28s
600:	learn: 15.3384372	total: 35.7s	remaining: 1m 23s
700:	learn: 14.6396543	total: 41.5s	remaining: 1m 16s
800:	learn: 14.1414590	total: 47.4s	remaining: 1m 10s
900:	learn: 13.7747720	total: 53.4s	remaining: 1m 5s
1000:	learn: 13.5223498	total: 59.3s	remaining: 59.1s
1100:	learn: 13.3500224	total: 1m 5s	remaining: 53.2s
1200:	learn: 13.2126256	total: 1m 10s	remaining: 47.2s
1300:	learn: 13.1151132	total: 1m 16s	remaining: 41.3s
1400:	learn: 13.0425634	total: 1m 22s	remaining: 35.4s
1500:	learn: 12.9857008	total: 1m 28s	remaining: 29.5s
1600:	learn: 12.9490163	total: 1m 34s	remaining: 23.6s
1700:	learn: 12.9160067	total: 1m 40s	remaining: 17.7s
1800:	learn: 12.8916296	tota

In [13]:
#Test de modèle sur données de test
X_test = X_test_df
predictions = final_model.predict(X_test)
print(predictions)

[344.67623948 293.62298392 203.41822699 207.31751095 232.88300967
 345.37776569 234.28696563 320.28931196 272.91118545 265.399382
 259.56596293 315.09634467 298.10907622 260.61628433 254.8556107
 595.47265494 290.53336446 311.63646513 360.25962281 151.89497078
 163.23331718 141.27085007 395.78629289 204.46793922 306.32596948
 172.04572202 642.34149322 463.44290678 427.61854118 321.40085228
 363.18129463 312.55343667 270.92662839 198.4886797  296.94088844
 259.15409068 348.37086596 282.79351215 232.97550836 345.63136188
 215.87539658 296.30725496 271.84472826 136.55685909 190.60475618
 279.68190884 344.07464041 248.79533883 290.64453585 348.10985352
 332.13573295 342.11176207 129.42482953 304.63053954 285.8293131
 286.42778516 222.22623164 266.0132557  277.74208545 242.58666673
 342.88991493 321.53964793 202.60756504 290.97328521 376.34274742
 200.77645105 266.06294801 332.90055775 279.58268724 309.19607075
 296.46361226 367.89758103 199.77701578 205.32955014 180.12813624
 290.78055008 

In [14]:
#Export au format csv
id = test_data_set['id']
Tm = predictions
dict = {"id": id, "Tm": Tm}
submission_df = pd.DataFrame(dict)

# Sauvegarder le DataFrame en fichier CSV
output_csv_path = "submission.csv"
submission_df.to_csv(output_csv_path, index=False, sep=',')

print(f"Fichier CSV généré : {output_csv_path}")
print(submission_df)

Fichier CSV généré : submission.csv
       id          Tm
0    1022  344.676239
1    1146  293.622984
2      79  203.418227
3    2279  207.317511
4    1342  232.883010
..    ...         ...
661  2663  277.849202
662   624  293.655730
663  2655  155.150454
664  2089  248.003594
665  1065  288.702335

[666 rows x 2 columns]


In [15]:
from IPython.display import FileLink
FileLink("submission.csv")

/kaggle/working/submission.csv